In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
from alphagogoat.embedder import process_battle, get_team_histories
from alphagogoat.catalogs import MoveEnum
from alphagogoat.delphox import Delphox, train
from alphagogoat.constants import LSTM_INPUT_SIZE, LSTM_OUTPUT_SIZE, device


from poke_env.environment.battle import Battle
from tqdm.auto import tqdm
import torch

import json
import logging
from copy import deepcopy
from pathlib import Path
from joblib import Parallel, delayed


LOGGER = logging.getLogger('poke-env')

In [47]:
def process_input_log(log):
    input_log = log['inputlog']
    input_log = input_log.split('\n')
    start = 0
    for line in input_log:
        if line.startswith('>p1'):
            break
        start += 1

    input_log = input_log[start:]
    out = []

    for i in range(len(input_log) - 1):
        curr_line = input_log[i]
        next_line = input_log[i+1]
        if curr_line.startswith('>p1') and next_line.startswith('>p2'): # that means this is a normal turn with no fainting or anything
            out_me = torch.zeros(len(MoveEnum) + 1)
            out_them = torch.zeros(len(MoveEnum) + 1)
            curr_line = curr_line.split(' ')
            next_line = next_line.split(' ')

            if curr_line[1] == 'move':
                out_me[MoveEnum[curr_line[2].lower()].value - 1] = 1
            elif curr_line[1] == 'switch':
                out_me[-1] = 1

            if next_line[1] == 'move':
                out_them[MoveEnum[next_line[2].lower()].value - 1] = 1
            elif next_line[1] == 'switch':
                out_them[-1] = 1
            i += 1
            out.append(torch.cat((out_me, out_them)))
        else:
            continue

    return out

def make_data(filepath):
    with open(filepath) as f:
        replay = json.load(f)
    history = replay['log'].split('\n')
    battles = []
    b = Battle(replay['id'], replay['p1'], LOGGER, 8)
    b._opponent_username = replay['p2']
    for line in history:
        try:
            b._parse_message(line.split('|'))
            if line.split('|')[1] == 'turn':
                battles.append(deepcopy(b))
        except:
            continue
    turn_actions = process_input_log(replay)
    h1, h2 = get_team_histories(battles)
    return h1, h2, turn_actions

In [48]:
json_files = [filepath for filepath in Path("cache/replays").iterdir() if filepath.name.endswith('.json')]
dataset = Parallel(n_jobs=4)(delayed(make_data)(filepath) for filepath in tqdm(json_files))

  0%|          | 0/145 [00:00<?, ?it/s]

2023-05-26 02:18:08,143 - poke-env - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p1a: Type: Null', 'Rest', 'p1a: Type: Null', '[from]Sleep Talk'] in battle gen8randombattle-1200357629 turn 14
2023-05-26 02:18:08,144 - poke-env - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p1a: Type: Null', 'Crush Claw', 'p2a: Ninetales', '[from]Sleep Talk'] in battle gen8randombattle-1200357629 turn 26
2023-05-26 02:18:08,144 - poke-env - WARNING - Unmanaged move message format received - cleaned up message ['', 'move', 'p1a: Type: Null', 'Swords Dance', 'p1a: Type: Null', '[from]Sleep Talk'] in battle gen8randombattle-1200357629 turn 31
2023-05-26 02:18:08,147 - poke-env - WARNING - Rating information regarding an unrecognized username received. Received 'kyuremwhite&gt;blick', while only known players are 'darkell8' and 'kyuremwhite>blick'


In [45]:
dataset[0]

[[], []]

In [ ]:
delphox = Delphox(LSTM_INPUT_SIZE, LSTM_OUTPUT_SIZE).to(device=device)
train(delphox, dataset)